In [4]:
# First cell - imports
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import requests
from datetime import datetime
import json

In [11]:
API_KEY = '68263b2623859b9550b9681663875ae2'
BASE_URL = 'https://api.stlouisfed.org/fred'

def get_fred_series(series_id, observation_start=None):
    """
    Fetch data series from FRED API
    
    Parameters:
    series_id (str): FRED series identifier
    observation_start (str): Start date in YYYY-MM-DD format
    
    Returns:
    pandas.DataFrame: Time series data
    """
    # Build API URL
    url = f"{BASE_URL}/series/observations"
    
    params = {
        'series_id': series_id,
        'api_key': API_KEY,
        'file_type': 'json',
        'observation_start': observation_start if observation_start else '1976-01-01'
    }
    
    # Make API request
    response = requests.get(url, params=params)
    data = response.json()
    
    # Convert to DataFrame
    df = pd.DataFrame(data['observations'])
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    return df.set_index('date')['value']

# Get our series
yield_spread = get_fred_series('T10Y2Y')
gdp = get_fred_series('GDPC1')
fed_funds = get_fred_series('DFF')
unenmployment = get_fred_series('UNRATE')

# Create main dataframe
df = pd.DataFrame({
    'yield_spread': yield_spread,
    'gdp': gdp,
    'fed_funds':fed_funds, 
    'unemployment': unenmployment
})

# Calculate GDP growth rate (annualized)
df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100

#export to excel to validate 
df.to_excel('stats.xlsx')

C:\Users\mrasm\AppData\Local\Temp\ipykernel_15672\551672040.py:51: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['gdp_growth'] = df['gdp'].pct_change(periods=4) * 100


Data Overview:


,yield_spread,gdp,fed_funds,unemployment,gdp_growth
date,,,,,
1976-07-01,0.86,6404.895,5.58,7.8,0.547408
1976-10-01,1.25,6451.177,5.32,7.7,0.722604
1977-04-01,1.41,6654.466,4.72,7.2,1.941924
1977-07-01,1.22,6774.457,5.36,6.9,1.803165
1979-10-01,-0.63,7318.535,12.06,6.0,0.250045



Summary Statistics:


,yield_spread,gdp,fed_funds,unemployment,gdp_growth
count,99.000000,99.000000,99.000000,99.000000,99.000000
mean,1.072121,13884.869949,4.494545,6.487879,0.685122
std,0.942129,4817.414400,4.585110,1.887352,1.337738
min,-1.900000,6404.895000,0.060000,3.500000,-7.908965
25%,0.385000,9432.769000,0.350000,5.200000,0.428404
50%,1.120000,14253.574000,3.360000,6.200000,0.771085
75%,1.830000,17599.761500,6.230000,7.350000,1.082608
max,2.840000,23386.248000,21.090000,14.800000,7.830225


In [13]:
# Show first few rows
display(df.head())

# Show basic statistics
display(df.describe())

,yield_spread,gdp,fed_funds,unemployment,gdp_growth
date,,,,,
1976-07-01,0.86,6404.895,5.58,7.8,0.547408
1976-10-01,1.25,6451.177,5.32,7.7,0.722604
1977-04-01,1.41,6654.466,4.72,7.2,1.941924
1977-07-01,1.22,6774.457,5.36,6.9,1.803165
1979-10-01,-0.63,7318.535,12.06,6.0,0.250045


,yield_spread,gdp,fed_funds,unemployment,gdp_growth
count,99.000000,99.000000,99.000000,99.000000,99.000000
mean,1.072121,13884.869949,4.494545,6.487879,0.685122
std,0.942129,4817.414400,4.585110,1.887352,1.337738
min,-1.900000,6404.895000,0.060000,3.500000,-7.908965
25%,0.385000,9432.769000,0.350000,5.200000,0.428404
50%,1.120000,14253.574000,3.360000,6.200000,0.771085
75%,1.830000,17599.761500,6.230000,7.350000,1.082608
max,2.840000,23386.248000,21.090000,14.800000,7.830225
